In [1]:
import dash
import dash_auth
import json
import dash_core_components as dcc
import dash_daq as daq
import dash_bootstrap_components as dbc
import dash_html_components as html
from dash.dependencies import Input, Output
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from itertools import cycle

import pandas as pd
import numpy as np
import datetime

In [2]:
from jupyter_dash import JupyterDash

In [3]:
opportunity = pd.read_csv('../data/days.csv', index_col=[0,1,2,3])
annual_operating = pd.read_csv('../data/annual.csv', index_col=[0,1])
stats = pd.read_csv('../data/scores.csv')
quantiles = np.arange(50,101,1)
quantiles = quantiles*.01
quantiles = np.round(quantiles, decimals=2)
lines = opportunity.index.get_level_values(1).unique()
asset_metrics = ['Yield', 'Rate', 'Uptime']
groupby = ['Line', 'Product group']
oee = pd.read_csv('../data/oee.csv')
oee['From Date/Time'] = pd.to_datetime(oee["From Date/Time"])
oee['To Date/Time'] = pd.to_datetime(oee["To Date/Time"])
oee["Run Time"] = pd.to_timedelta(oee["Run Time"])
oee = oee.loc[oee['Rate'] < 2500]
res = oee.groupby(groupby)[asset_metrics].quantile(quantiles)

df = pd.read_csv('../data/products.csv')
descriptors = df.columns[:8]
production_df = df
production_df['product'] = production_df[descriptors[2:]].agg('-'.join, axis=1)
production_df = production_df.sort_values(['Product Family', 'EBIT'],
                                          ascending=False)

stat_df = pd.read_csv('../data/category_stats.csv')
old_products = df[descriptors].sum(axis=1).unique().shape[0]
weight_match = pd.read_csv('../data/weight_match.csv')

In [4]:
weight_match

,Line,Product Family,Base Polymer,Base Type,Additional Treatment,Width Material Attri,Thickness Material A,Quantity Good,Run Time Hours,Net Quantity Produced,...,Adjusted EBITDA,Total Depreciation,EBIT,Product,Weight Adjusted EBITDA,Rate,Yield,EBITDA per Hr,EBITDA Rank,EBITDA per Hr Rank
0,K06,Shrink Sleeve,PVC,148/07,T35,250,45,1169.998,0.871389,1169.998,...,-6064.99,1092.77,-7157.76,PVC-148/07-T35-GC,-869.216920,1342.681798,1.000000,-6960.141536,0,834
1,K18,Cards Overlay,PVC,278/01,SB6,1050,50,1058.000,1.746944,1058.000,...,-9271.02,1904.17,-11175.19,PVC-278/01-SB6-GC,-1635.062370,605.628876,1.000000,-5306.991891,1,833
2,K06,Shrink Sleeve,PVC,148/05,T30,355,50,1772.000,1.415000,1772.000,...,-6371.88,1332.49,-7704.37,PVC-148/05-T30-GC,-1078.349986,1252.296820,1.000000,-4503.095406,2,832
3,K06,Shrink Sleeve,PVC,147/07,T25,640,45,2217.997,1.778611,2217.997,...,-7917.56,1420.30,-9337.86,PVC-147/07-T25-GC,-1650.543660,1247.038763,1.000000,-4451.540840,3,831
4,K06,Shrink Sleeve,PVC,145/01,T02,840,40,3655.995,2.275833,3655.995,...,-9545.89,1049.15,-10595.04,PVC-145/01-T02-GC,-4298.525201,1606.442329,1.000000,-4194.459172,4,830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,E27,Shrink Sleeve,PET,G10F22,T38,470,45,3700.000,2.341389,3700.000,...,12149.87,1679.33,10470.54,PETG-G10F22-T38-GC,3352.663142,1580.258631,1.000000,5189.172144,829,5
830,E27,Shrink Sleeve,PET,G10F22,T52,800,50,2454.983,1.433889,2454.983,...,8146.41,1108.48,7037.93,PETG-G10F22-T52-GC,2909.242706,1712.115227,1.000000,5681.339791,830,4
831,E27,Shrink Sleeve,PET,G10F22,T52,416,45,1964.398,0.947778,1964.398,...,5395.30,1455.12,3940.18,PETG-G10F22-T52-GC,1080.797509,2072.635639,1.000000,5692.579132,831,3
832,E27,Shrink Sleeve,PET,G10F22,T38,480,45,1466.700,0.946111,1466.700,...,5882.82,933.05,4949.77,PETG-G10F22-T38-GC,1176.676317,1550.240752,1.000000,6217.895479,832,2


In [5]:
stat_df

,descriptor,group,stat,p,m,table,score
0,Thickness Material A,47,17.162393,3.431632e-05,-114.56,[[ 0 1321]\n [ 19 1303]],0.000000
1,Base Type,148/07,102.065139,5.372574e-24,-114.56,[[ 4 1317]\n [ 111 1211]],0.036036
2,Base Type,148/05,70.735420,4.084922e-17,-114.56,[[ 3 1318]\n [ 79 1243]],0.037975
3,Base Type,419/58,17.528715,2.830011e-05,-114.56,[[ 1 1320]\n [ 22 1300]],0.045455
4,Base Type,C8CF03,12.574290,3.910898e-04,-114.56,[[ 1 1320]\n [ 17 1305]],0.058824
5,Additional Treatment,T25,104.021622,2.001202e-24,-114.56,[[ 16 1305]\n [ 141 1181]],0.113475
6,Additional Treatment,T00,8.516289,3.519816e-03,-114.56,[[ 2 1319]\n [ 15 1307]],0.133333
7,Base Type,419/59,8.504522,3.542650e-03,-114.56,[[ 3 1318]\n [ 17 1305]],0.176471
8,Base Type,230/58,31.503208,1.991111e-08,-114.56,[[ 12 1309]\n [ 60 1262]],0.200000
9,Color Group,TR,50.566130,1.152164e-12,-114.56,[[ 19 1302]\n [ 94 1228]],0.202128


In [6]:
df

,Product Family,Product Group,Base Polymer,Base Type,Additional Treatment,Color Group,Width Material Attri,Thickness Material A,Sales Quantity in KG,Net Sales Quantity in KG,...,Corporate Allocated Costs Prod,Contribution Margin II - C,Research & Development,Selling exp. (Condition),"General, & Administrative (Condition)",EBITDA before Allocations,Adjusted EBITDA,Total Depreciation,EBIT,product
0,Cards Core,Cards Core,PETG,G01T01,ZZZ,WH,468,240,22413.0,22413.0,...,507.61,15401.24,829.28,3912.86,1409.55,9249.55,9249.55,1502.21,7747.34,PETG-G01T01-ZZZ-WH-468-240
1,Cards Core,Cards Core,PVC,146/19,ZZZ,BK,310,320,1893.0,1893.0,...,38.79,2179.78,70.04,330.48,119.05,1660.21,1660.21,59.12,1601.09,PVC-146/19-ZZZ-BK-310-320
2,Cards Core,Cards Core,PVC,146/19,ZZZ,BK,475,380,4021.0,4021.0,...,98.17,2877.48,148.78,701.99,252.88,1773.83,1773.83,134.26,1639.57,PVC-146/19-ZZZ-BK-475-380
3,Cards Core,Cards Core,PVC,146/19,ZZZ,BK,510,320,2034.0,2034.0,...,39.01,3670.33,75.26,355.10,127.92,3112.05,3112.05,63.43,3048.62,PVC-146/19-ZZZ-BK-510-320
4,Cards Core,Cards Core,PVC,146/19,ZZZ,BK,586,310,22650.0,22650.0,...,349.08,27791.19,838.05,3954.24,1424.45,21574.45,21574.45,593.94,20980.51,PVC-146/19-ZZZ-BK-586-310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2638,Tape,Tape,PVC,240/15,ZZZ,TL,2120,31,1128349.1,1128349.1,...,25371.41,-103160.73,41748.90,97241.13,70961.87,-313112.63,-313112.63,35861.05,-348973.68,PVC-240/15-ZZZ-TL-2120-31
2639,Tape,Tape,PVC,240/15,ZZZ,TP,100,40,5606.5,5606.5,...,158.63,16469.62,207.44,483.17,352.59,15426.42,15426.42,520.63,14905.79,PVC-240/15-ZZZ-TP-100-40
2640,Tape,Tape,PVC,240/15,ZZZ,TP,1280,70,2973.0,2973.0,...,64.04,4719.04,110.00,256.21,186.97,4165.86,4165.86,103.10,4062.76,PVC-240/15-ZZZ-TP-1280-70
2641,Tape,Tape,PVC,240/91,ZZZ,TL,1280,38,14560.1,14560.1,...,276.51,3291.97,538.74,1254.79,915.68,582.76,582.76,475.21,107.55,PVC-240/91-ZZZ-TL-1280-38


In [7]:
oee

,FA,Material,From Date/Time,To Date/Time,Net Quantity Produced,Quantity Good,Product group,Polymer,Line,Run Time,Yield,Rate,different start end days,Uptime,Thickness Material A,Width Material Attri,Base Type,Additional Treatment,Product,Shift
0,100779744,LF-TG10F22-11G0000-100BJ-T52 - 50-4300R,2018-12-30 21:35:36,2019-01-02 02:22:00,11762.078,11206.578,Shrink Sleeve,PET,E26,2 days 04:46:24,0.952772,222.879194,True,NaN,50,300,G10F22,T52,PET-G10F22-T52,B
1,100778032,LF-TG10F22-11G0000-100BJ-T52 - 45-4300R,2019-01-02 02:22:00,2019-01-03 02:46:14,30060.155,29419.000,Shrink Sleeve,PET,E26,1 days 00:24:14,0.978671,1231.777244,True,0.0,45,300,G10F22,T52,PET-G10F22-T52,C
2,100780338,LF-TG10F22-11G0000-100BJ-T52 - 45- 690R,2019-01-03 02:46:14,2019-01-03 05:47:25,3736.026,3456.026,Shrink Sleeve,PET,E26,0 days 03:01:11,0.925054,1237.208500,False,0.0,45,690,G10F22,T52,PET-G10F22-T52,C
3,100780337,LF-TG10F22-11G0000-100BJ-T52 - 45- 675R,2019-01-03 05:47:25,2019-01-03 08:39:16,3602.026,3482.026,Shrink Sleeve,PET,E26,0 days 02:51:51,0.966685,1257.617457,False,0.0,45,675,G10F22,T52,PET-G10F22-T52,A
4,100780307,LF-TG10F22-11G0000-100BJ-T52 - 40- 711R,2019-01-03 08:39:16,2019-01-03 14:38:23,6481.000,5988.000,Shrink Sleeve,PET,E26,0 days 05:59:07,0.923931,1082.823595,False,0.0,40,711,G10F22,T52,PET-G10F22-T52,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5217,100904769,FF-M225/05-95/2500-C75_0-ZZZ - 160-1430R,2019-12-16 15:29:10,2019-12-17 00:01:00,9335.000,8885.000,Dec. Surfaces/others,PVC,K40,0 days 08:31:50,0.951794,1094.301530,True,0.0,160,430,225/05,ZZZ,PVC-225/05-ZZZ,B
5218,100908556,FF-M225/05-95/2500-B68_0-ZZZ - 140- 218R,2019-12-17 00:01:00,2019-12-17 02:01:00,2582.000,2017.000,Dec. Surfaces/others,PVC,K40,0 days 02:00:00,0.781177,1291.000000,False,0.0,140,218,225/05,ZZZ,PVC-225/05-ZZZ,B
5219,100908557,FF-M225/05-95/2500-B68_0-ZZZ - 140-1500R,2019-12-17 03:31:46,2019-12-17 04:15:00,583.100,583.100,Dec. Surfaces/others,PVC,K40,0 days 00:43:14,1.000000,809.236700,False,0.0,140,500,225/05,ZZZ,PVC-225/05-ZZZ,B
5220,100909646,FF-M225/05-95/2500-B68_0-ZZZ - 150-1470R,2019-12-17 04:15:00,2019-12-17 06:25:05,2287.000,2287.000,Dec. Surfaces/others,PVC,K40,0 days 02:10:05,1.000000,1054.862268,False,0.0,150,470,225/05,ZZZ,PVC-225/05-ZZZ,A


In [8]:
stats

,Unnamed: 0,descriptor,group,stat,p,m,table,score,metric
0,0,Product group,PSL,60.969141,5.797648e-15,0.000000,[[ 23 469]\n [ 34 4695]],1.478261,Uptime
1,1,Line,E28,87.355919,9.064846e-21,0.000000,[[ 33 459]\n [ 50 4679]],1.515152,Uptime
2,2,Line,E26,80.300202,3.216362e-19,0.000000,[[ 70 422]\n [ 213 4516]],3.042857,Uptime
3,3,Product group,Shrink Sleeve,44.097937,3.123500e-11,0.000000,[[ 232 260]\n [1522 3207]],6.560345,Uptime
4,4,Line,E27,7.367392,6.641720e-03,0.000000,[[ 88 404]\n [ 631 4098]],7.170455,Uptime
5,5,Line,K18,7.799445,5.226228e-03,0.000000,[[ 69 423]\n [ 913 3816]],13.231884,Uptime
6,6,Line,K17,6.898660,8.626038e-03,0.000000,[[ 28 464]\n [ 443 4286]],15.821429,Uptime
7,7,Product group,Cards Overlay,7.502826,6.160225e-03,0.000000,[[ 19 473]\n [ 344 4385]],18.105263,Uptime
8,8,Product group,Dec. Surfaces/others,49.903831,1.614691e-12,0.000000,[[ 72 420]\n [1411 3318]],19.597222,Uptime
9,9,Line,K40,48.941238,2.637468e-12,0.000000,[[ 59 433]\n [1252 3477]],21.220339,Uptime


In [9]:
annual_operating

Net Quantity Produced     Run Time  Quantity Good
Line Product group                                                          
E26  Dec. Surfaces/others             331666.915  2158.999722     185956.944
     Shrink Sleeve                   3121577.978  4885.885833    2721629.545
E27  Shrink Sleeve                   9527580.297  8403.696111    9460495.215
E28  PH Mono Films                      6393.556     4.168056        586.604
     PSL                             1280861.189  3994.988333    1023616.183
     Roll Sleeve                        4082.866    10.926111       1194.544
     Shrink Sleeve                    616467.246  1029.150556     577362.094
K06  Shrink Sleeve                   6563709.072  5801.175833    6253266.484
K10  Cards Core                      3999948.812  4388.537500    3616310.225
     Construction                    1983522.071  1588.302778    1869041.878
     Cooling Tower                    236971.700   187.375278     206396.200
     Dec. Surfaces/others              63560.308    74.371389      57711.500
     Other Technical                   65847.900    71.848611      64312.900
     Print                             27312.000    25.390278      21429.000
K17  Tape                            9478328.700  9464.006111    9023290.700
K18  Cards Core                      1026021.300   747.331389     919097.700
     Cards Overlay                   1271407.600  1500.369444    1198156.600
     Construction                     744636.712   440.958333     700473.412
     Cooling Tower                   2293338.154  1180.929722    2212580.154
     Dec. Surfaces/others            1967730.200  2256.736389    1809294.200
     Other Technical                  104935.000    57.363333      93496.000
     Print                             79143.000    47.911667      71432.000
K40  Cards Core                        95718.200    93.515556      83124.300
     Dec. Surfaces/others            5821181.000  5881.121944    5363693.500
     Roll Sleeve                        8942.100    17.673333       6553.000
     Shrink Sleeve                    812588.800   756.111667     742929.000
     Stationery                        26043.100    41.387778      21700.000
     Tape                              34033.600    40.198889      30692.000

In [10]:
opportunity

Rate      Yield  \
Basis           Line      Product group                                 
Additional Days E26  0.50 Dec. Surfaces/others    0.000000   0.000000   
                          Shrink Sleeve           0.000000   0.000000   
                     0.51 Dec. Surfaces/others    0.224770   0.956659   
                          Shrink Sleeve           0.619090   0.433102   
                     0.52 Dec. Surfaces/others    0.314963   1.114807   
...                                                    ...        ...   
                K40  1.00 Dec. Surfaces/others  173.146636  15.810691   
                          Roll Sleeve             0.039294   0.000000   
                          Shrink Sleeve           5.888814   2.349726   
                          Stationery              0.199387   0.084804   
                          Tape                    0.409578   0.073526   

                                                   Uptime  
Basis           Line      Product group                    
Additional Days E26  0.50 Dec. Surfaces/others   0.000000  
                          Shrink Sleeve          0.000000  
                     0.51 Dec. Surfaces/others   0.000000  
                          Shrink Sleeve          0.000000  
                     0.52 Dec. Surfaces/others   0.000000  
...                                                   ...  
                K40  1.00 Dec. Surfaces/others  10.624884  
                          Roll Sleeve            0.000000  
                          Shrink Sleeve          4.017917  
                          Stationery             9.933669  
                          Tape                   5.120035  

[1428 rows x 3 columns]

In [11]:
def make_violin_plot(sort='Worst', select=[0,10], descriptors=None):

    if sort == 'Best':
        local_df = stat_df.sort_values('score', ascending=False)
        local_df = local_df.reset_index(drop=True)
    else:
        local_df = stat_df
    if descriptors != None:
        local_df = local_df.loc[local_df['descriptor'].isin(descriptors)]
    fig = go.Figure()
    for index in range(select[0],select[1]):
        x = df.loc[(df[local_df.iloc[index]['descriptor']] == \
            local_df.iloc[index]['group'])]['Adjusted EBITDA']
        y = local_df.iloc[index]['descriptor'] + ': ' + df.loc[(df[local_df\
            .iloc[index]['descriptor']] == local_df.iloc[index]['group'])]\
            [local_df.iloc[index]['descriptor']]
        name = '€ {:.0f}'.format(x.median())
        fig.add_trace(go.Violin(x=y,
                                y=x,
                                name=name,
                                box_visible=True,
                                meanline_visible=True))
    fig.update_layout({
                "plot_bgcolor": "#F9F9F9",
                "paper_bgcolor": "#F9F9F9",
                "title": 'Adjusted EBITDA by Product Descriptor (Median in Legend)',
                "yaxis.title": "EBITDA (€)",
                "height": 400,
                "margin": dict(
                       l=0,
                       r=0,
                       b=0,
                       t=30,
                       pad=4
   ),
                })

    return fig

def make_sunburst_plot(clickData=None, toAdd=None, col=None, val=None):
    if clickData != None:
        col = clickData["points"][0]['x'].split(": ")[0]
        val = clickData["points"][0]['x'].split(": ")[1]
    elif col == None:
        col = 'Thickness Material A'
        val = '47'

    desc = list(descriptors[:-2])
    if col in desc:
        desc.remove(col)
    if toAdd != None:
        for item in toAdd:
            desc.append(item)
    test = production_df.loc[production_df[col] == val]
    fig = px.sunburst(test, path=desc[:], color='Adjusted EBITDA', title='{}: {}'.format(
        col, val),
        color_continuous_scale=px.colors.sequential.Viridis
                     )
    fig.update_layout({
                "plot_bgcolor": "#F9F9F9",
                "title": '(Select in Violin) {}: {}'.format(col,val),
                "paper_bgcolor": "#F9F9F9",
                "height": 400,
                "margin": dict(
                       l=0,
                       r=0,
                       b=0,
                       t=30,
                       pad=4
   ),
                })
    return fig

def make_ebit_plot(production_df, select=None, sort='Worst', descriptors=None):
    families = production_df['Product Family'].unique()
    colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3',\
              '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
    colors_cycle = cycle(colors)
    grey = ['#7f7f7f']
    color_dic = {'{}'.format(i): '{}'.format(j) for i, j  in zip(families,
                                                                 colors)}
    grey_dic =  {'{}'.format(i): '{}'.format('#7f7f7f') for i in families}
    fig = go.Figure()


    if select == None:
        for data in px.scatter(
                production_df,
                x='product',
                y='Adjusted EBITDA',
                color='Product Family',
                color_discrete_map=color_dic,
                opacity=1).data:
            fig.add_trace(
                data
            )

    elif select != None:
        color_dic = {'{}'.format(i): '{}'.format(j) for i, j  in zip(select,
                                                                     colors)}
        for data in px.scatter(
                production_df,
                x='product',
                y='Adjusted EBITDA',
                color='Product Family',

                color_discrete_map=color_dic,
                opacity=0.09).data:
            fig.add_trace(
                data
            )

        if sort == 'Best':
            local_df = stat_df.sort_values('score', ascending=False)
        elif sort == 'Worst':
            local_df = stat_df


        new_df = pd.DataFrame()
        if descriptors != None:
            local_df = local_df.loc[local_df['descriptor'].isin(descriptors)]
        for index in select:
            x = production_df.loc[(production_df[local_df.iloc[index]\
                ['descriptor']] == local_df.iloc[index]['group'])]
            x['color'] = next(colors_cycle) # for line shapes
            new_df = pd.concat([new_df, x])
            new_df = new_df.reset_index(drop=True)
        for data in px.scatter(
                new_df,
                x='product',
                y='Adjusted EBITDA',
                color='Product Family',

                color_discrete_map=color_dic,
                opacity=1).data:
            fig.add_trace(
                data
            )
        shapes=[]

        for index, i in enumerate(new_df['product']):
            shapes.append({'type': 'line',
                           'xref': 'x',
                           'yref': 'y',
                           'x0': i,
                           'y0': -4e5,
                           'x1': i,
                           'y1': 4e5,
                           'line':dict(
                               dash="dot",
                               color=new_df['color'][index],)})
        fig.update_layout(shapes=shapes)
    fig.update_layout({
            "plot_bgcolor": "#F9F9F9",
            "paper_bgcolor": "#F9F9F9",
            "title": 'Adjusted EBITDA by Product Family',
            "yaxis.title": "EBITDA (€)",
            "height": 500,
            "margin": dict(
                   l=0,
                   r=0,
                   b=0,
                   t=30,
                   pad=4
),
            "xaxis.tickfont.size": 8,
            # "font":dict(
            #     size=8,
            # ),
            })
    return fig

In [12]:
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
html.Div([
    html.Div([
        html.H3(["Product Margin Optimization"]),
        ], className='nine columns',
        ),
], className='row flex-display',
),
html.Div([
html.Div([
dcc.Markdown('''
###### Key Finding: ######
There are a fair number of low to negative margin products
that should be reviewed. All groups &#150 business, manufacturing, supply
chain &#150 need to work together to improve these margins by using a combination
of potential levers: Price Increase, Production Rules, Minimum Order Sizes, Campaigning, etc.

**Implementation Phase:** Caravel partners work with group teams to strategize
products around margin levers.

### Est. Impact € 3.5-6 M/Yr ###
'''),
], className='pretty_container',
   style={"background-color": "#ffffff",
          "maxHeight": "800px"},
    id='explain1a',
),
html.Div([
dcc.Markdown('''

###### Demonstrates margin disparity and product buckets. ######

This analysis starts with finding key product descriptors that affect EBITDA
 using a priopriatary statistical test. These can be selected by the range bar in the side panel, which
can be sorted by whether they correlate with high or low EBITDA.
 A violin plot of EBITDA values is constructed of each descriptor which is a method of plotting
distributions. It is similar to a box plot, with the addition of a rotated
kernel density (kde) plot on each side. **The benefit of the kde is to visualize
the density of the data without obstructing key outliers** *(ex: 200-400K EBITDA
outliers in 2D Coil Coating and Base Type 153/07)*

Clicking on a distribution in the violin
plot expands the sunburst chart to its right. A sunburst chart is a way of
representing hierarchical data structures. In this case it is showing the
product breakdown for a given descriptor. For instance, products with base
types of 202/14 fall within the Construction category, with PVC polymer, ZZZ
treatment, and OP color. The bandwidths that lie on each ring indicate the
production volume fraction for that given descriptor while color indicates
the average EBITDA for all products described by that section of the sunburst *(ex:
in the default view, highest EBITDA base type 202/14 products have a width of 955
while lowest EBITDA have a width of 400 and each of these count for 1 production
run out of 23 for this product group).* Thickness and width can be toggled on the sunburst chart for clarity.

Descriptors in the violin plot are overlayed onto the EBITDA by Product Family
chart. In this way, product descriptors can be evaluated within the broader portfolio
*(ex: toggling the best/worst rank selector above
will alternate highlighting the high margin and negative margin products within
each family, respectively).*
'''),
], className='pretty_container',
   style={"background-color": "#ffffff",
          "maxHeight": "800px"},
   id='explain1b',
),
], className='row container-display',
),
    html.Div([
        html.Div([
            html.H6(id='margin-new-rev'), html.P('Adjusted EBITDA')
        ], className='mini_container',
           id='margin-rev',

        ),
        html.Div([
            html.H6(id='margin-new-rev-percent'), html.P('Unique Products')
        ], className='mini_container',
           id='margin-rev-percent',
        ),
        html.Div([
            html.H6(id='margin-new-products'), html.P('Volume')
        ], className='mini_container',
           id='margin-products',
        ),
    ], className='row container-display',
    ),
    html.Div([
        html.Div([
            html.P('Descriptors'),
            dcc.Dropdown(id='descriptor_dropdown',
                         options=[{'label': 'Thickness', 'value': 'Thickness Material A'},
                                 {'label': 'Width', 'value': 'Width Material Attri'},
                                 {'label': 'Base Type', 'value': 'Base Type'},
                                 {'label': 'Additional Treatment', 'value': 'Additional Treatment'},
                                 {'label': 'Color', 'value': 'Color Group'},
                                 {'label': 'Product Group', 'value': 'Product Group'},
                                 {'label': 'Base Polymer', 'value': 'Base Polymer'},
                                 {'label': 'Product Family', 'value': 'Product Family'}],
                         value=['Thickness Material A',
                                'Width Material Attri', 'Base Type',
                                'Additional Treatment', 'Color Group',
                                'Product Group',
                                'Base Polymer', 'Product Family'],
                         multi=True,
                         className="dcc_control"),
            html.P('Number of Descriptors:', id='descriptor-number'),
            dcc.RangeSlider(
                        id='select',
                        min=0,
                        max=stat_df.shape[0],
                        step=1,
                        value=[0,10],
            ),
            html.P('Sort by:'),
            dcc.RadioItems(
                        id='sort',
                        options=[{'label': i, 'value': j} for i, j in \
                                [['Low EBITDA', 'Worst'],
                                ['High EBITDA', 'Best']]],
                        value='Best',
                        labelStyle={'display': 'inline-block'},
                        style={"margin-bottom": "10px"},),
            html.P('Toggle Violin/Descriptor Data onto EBITDA by Product Family:'),
            daq.BooleanSwitch(
              id='daq-violin',
              on=False,
              style={"margin-bottom": "10px", "margin-left": "0px",
              'display': 'inline-block'}),
                ], className='mini_container',
                    id='descriptorBlock',
                ),
            html.Div([
                dcc.Graph(
                            id='ebit_plot',
                            figure=make_ebit_plot(production_df)),
                ], className='mini_container',
                   id='ebit-family-block'
                ),
        ], className='row container-display',
        ),

    html.Div([
        html.Div([
            dcc.Graph(
                        id='violin_plot',
                        figure=make_violin_plot()),
                ], className='mini_container',
                   id='violin',
                ),
        html.Div([
            dcc.Dropdown(id='length_width_dropdown',
                        options=[{'label': 'Thickness', 'value': 'Thickness Material A'},
                                 {'label': 'Width', 'value': 'Width Material Attri'}],
                        value=['Width Material Attri'],
                        multi=True,
                        placeholder="Include in sunburst chart...",
                        className="dcc_control"),
            dcc.Graph(
                        id='sunburst_plot',
#                         figure=make_sunburst_plot()
            ),
                ], className='mini_container',
                   id='sunburst',
                ),
            ], className='row container-display',
               style={'margin-bottom': '50px'},
            ),
], className='pretty container'
)

In [14]:
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
